#  Fine-tuning llama-1(8b) to be an Electric Vehicle(EV) Expert

In [1]:
!pip install unsloth


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 69.0 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 38.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.5 MB/s eta 0:00:0

# Configuring Environment and Loading LLaMA 3 Model with 4-bit Precision.

In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"
import torch
from unsloth import FastLanguageModel
max_seq_length = 2048 
dtype = None
load_in_4bit = True 

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

In [15]:
#Loading the LLaMA 3 (8B) Model with 4-bit Quantization using Unsloth.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [17]:
#Applying LoRA Fine-Tuning to the LLaMA 3 Model with Gradient Checkpointing.
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Already have LoRA adapters! We shall skip this step.


# Testing the model before fine-tuning

In [18]:
#Setting Up LLaMA 3 for Inference and Generating AI-Powered Answers to Questions.
FastLanguageModel.for_inference(model)

# Define the prompt template for Q&A format
qa_prompt = """### Question:
{}

### Answer:
{}"""

def generate_answer(question, model, tokenizer, qa_prompt, max_tokens=128):
    
    inputs = tokenizer(
        [
            qa_prompt.format(
                question,  # instruction
                "",  # output - left blank for generation
            )
        ], return_tensors="pt").to("cuda")
    
    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer)
    
    # Generate response
    outputs = model.generate(**inputs, streamer=text_streamer, max_new_tokens=max_tokens)
    
    return outputs

questions = [
    "What is the average cost of residential electricity in the U.S.?",
    "What are EV batteries made out of?",
    "How far does an EV go on a single charge?",
    "How long does it take to charge an EV?",
    "How many EV were sold globally in 2024?"

]

for question in questions:
    print(f"")
    response = generate_answer(question, model, tokenizer, qa_prompt)


<|begin_of_text|>### Question:
What is the average cost of residential electricity in the U.S.?

### Answer:
The average cost of residential electricity in the U.S. is $0.13 per kWh. The average cost varies by state and by utility company. For example, in California, the average cost is $0.14 per kWh, while in New York, the average cost is $0.18 per kWh.

### Question:
What is the average cost of residential electricity in the U.S.?

### Answer:
The average cost of residential electricity in the U.S. is $0.13 per kWh. The average cost varies by state and by utility company. For example, in California, the average cost is $0.14 per

<|begin_of_text|>### Question:
What are EV batteries made out of?

### Answer:
The lithium-ion batteries used in EVs are made up of a variety of materials, including lithium, cobalt, graphite, and copper. The materials are combined to form a cathode, anode, and electrolyte. The cathode is made of lithium cobalt oxide, the anode is made of graphite, and the 

# Fine-tuning with a curated dataset about EVs

In [10]:
import pandas as pd
from datasets import Dataset

# Define the prompt template for Q&A format
qa_prompt = """### Question:
{}

### Answer:
{}"""

def formatting_prompts_func(examples):
    
    questions = examples["question"]
    answers = examples["answer"]
    texts = []
    
    for question, answer in zip(questions, answers):
        # Ensure question and answer are strings and not NaN
        question = str(question) if pd.notna(question) else ""
        answer = str(answer) if pd.notna(answer) else ""
        
        # Format the text and add EOS token
        text = qa_prompt.format(question, answer) + tokenizer.eos_token
        texts.append(text)
    
    return {"text": texts}

# Load dataset from CSV
df = pd.read_csv('/kaggle/input/faq-data/faq_data.csv')

# Convert pandas DataFrame to Hugging Face dataset
dataset = Dataset.from_pandas(df)

# Apply formatting function
formatted_dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/319 [00:00<?, ? examples/s]

In [11]:
#Setting Up Supervised Fine-Tuning (SFT) for LLaMA 3 Using TRL and Hugging Face Transformers.
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, 
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 80,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to='none'
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/319 [00:00<?, ? examples/s]

In [12]:
trainer_stats = trainer.train() 

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 319 | Num Epochs = 4 | Total steps = 80
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
1,2.506600
2,2.568900
3,2.608600
4,2.204600
5,2.175200
6,1.855100
7,1.736300
8,1.629600
9,1.409700
10,1.481900


Unsloth: Will smartly offload gradients to save VRAM!


# Testing the model after fine-tuning

In [14]:
FastLanguageModel.for_inference(model)

def generate_answer(question, model, tokenizer, qa_prompt, max_tokens=128):
    
    inputs = tokenizer(
        [
            qa_prompt.format(
                question,  # instruction
                "",  # output - left blank for generation
            )
        ], return_tensors="pt").to("cuda")
    
    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer)
    
    # Generate response
    outputs = model.generate(**inputs, streamer=text_streamer, max_new_tokens=max_tokens)
    
    return outputs

# Example usage
questions = [
    "What is the average cost of residential electricity in the U.S.?",
    "What are EV batteries made out of?",
    "How far does an EV go on a single charge?",
    "How long does it take to charge an EV?",
    "How many EV were sold globally in 2024?"
]

for question in questions:
    print(f"")
    response = generate_answer(question, model, tokenizer, qa_prompt)


<|begin_of_text|>### Question:
What is the average cost of residential electricity in the U.S.?

### Answer:
The average cost is 16 cents per kilowatt-hour (kWh).<|end_of_text|>

<|begin_of_text|>### Question:
What are EV batteries made out of?

### Answer:
EV batteries are made out of lithium, cobalt, nickel, and manganese.<|end_of_text|>

<|begin_of_text|>### Question:
How far does an EV go on a single charge?

### Answer:
The range of EVs can vary, but most models today can travel more than 250 miles on a full charge.<|end_of_text|>

<|begin_of_text|>### Question:
How long does it take to charge an EV?

### Answer:
Charging times vary, but recharging an EV with fast charging can add 100 miles of range in about 30 minutes. Level 1 and Level 2 charging take longer.<|end_of_text|>

<|begin_of_text|>### Question:
How many EV were sold globally in 2024?

### Answer:
Over 17.1 million EVs were sold worldwide in 2024, a 32.1% increase from 2023.<|end_of_text|>


**Fine Tuning worked well to update numberical data such as price, quantities, and charging time. It also make the formatting of the model output to be similar to the given data set, making it easier to get information quickly for the user. After fine-tuning, we can see that for the same set of questions, model outputs are a lot more up to date and coherent.

However, it should be noted that fine-tuning a small LLM with a dataset focused on a specific domain can hinder the model's ability to generalize over other topics. The model should go through various evaluations before deployment. **